# Uniview module for SN 1987a

*Aaron Geller, 2018*

### Imports and function definitions

In [15]:
#This directory contains all the data needed for the module. It should be in the same directory as the notebook
dataFolder = "data" 
import sys, os, shutil, errno, string, urllib


sys.path.append(( os.path.abspath( os.path.join(os.path.realpath("__file__"),  os.pardir, os.pardir) )))
import uvmodlib.v1 as uvmod

### USES Conf Template 

In [42]:
Template = """mesh
{  
    data center    ./modules/$folderName/center.raw
    data sphere    ./modules/$folderName/sphere.raw
    data  quad ./modules/$folderName/quad.3ds
    
    cullRadius $cr
    glslVersion 330
    
    propertyCollection 
    {        
        __objectName__
        {           

            vec1f SNRadMax 25 | public | desc "max SN radius" | widget slider | range 0 100
            vec1f SNTimeMax 10 | public | desc "max SN time" | widget slider | range 0 30
            vec1f SNTimeMin 3 | public | desc "start SN time" | widget slider | range 0 30
            vec1f SNAlpha 1 | public | desc "SN alpha " | widget slider | range 0 1
            vec1f SNFadeFac 100 | public | desc "SN fade factor " | widget slider | range 0 100
            vec1f neutrinoRadMax 10 | public | desc "max neutrino radius" | widget slider | range 0 500
            vec1f neutrinoTimeMax 2 | public | desc "max neutrino time" | widget slider | range 0 30
            vec1f neutrinoAlpha 1 | public | desc "neutrino alpha " | widget slider | range 0 1
            vec1f neutrinoColorP 0.1  | public | desc "neutrino color along colormap" | widget slider | range 0 1
            vec1f neutrinoLength 1.5  | public | desc "neutrino color along colormap" | widget slider | range 1 4
            
            vec1f eventTime 0 | public | desc "event time " #| widget slider | range 0 30
            vec1f transitionLength 5 | public | desc "transition length in seconds" 
            bool jump true | public  | desc "jump to time without transition"          
            
        }
    }
    
    ############# to hold the time information
    renderTexture
    {
        name                      stateTexture
        width                     1
        height                    1
        numTextures               1
        isPingPong                true
        isPersistent              true
        isFramePersistent         true
        internalTextureFormat     GL_RGB32F
        magnify                   GL_NEAREST
        minify                    GL_NEAREST
    }
    ############# set Transition State
    pass 
    {
        useDataObject quad
        renderTarget
        {
            name stateTexture
            enableColorClear false
        }
         shader
        {
            type defaultMeshShader
            {
                vertexShader   ./modules/$folderName/pass0.vs
                fragmentShader ./modules/$folderName/state.fs
                textureFBO stateTexture stateTexture
                stateManagerVar __objectName__.transitionLength transitionLength 
                stateManagerVar __objectName__.jump jump 
                stateManagerVar __objectName__.eventTime eventTime 
                parameter2f timeRange 0 100
           }
        }      
    }
    
    
    ############# Neutrino explosion
    pass
    {
        useDataObject sphere
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/neutrino.gs
                vertexShader ./modules/$folderName/SN.vs
                fragmentShader   ./modules/$folderName/neutrino.fs
                textureFBO stateTexture stateTexture
                texture cmap ./modules/$folderName/cmapNS.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                } 
                
                stateManagerVar __objectName__.neutrinoRadMax  neutrinoRadMax
                stateManagerVar __objectName__.neutrinoTimeMax  neutrinoTimeMax
                stateManagerVar __objectName__.neutrinoColorP  neutrinoColorP
                stateManagerVar __objectName__.neutrinoAlpha  neutrinoAlpha
                stateManagerVar __objectName__.neutrinoLength  neutrinoLength
                #stateManagerVar __objectName__.eventTime  eventTime
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE false
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                    UV_LINE_WIDTH 4
                }
            }

        }    
    }
    
    ############# Optical explosion
    pass
    {
        useDataObject center
        shader
        {
            type defaultMeshShader
            {
                geometryShader   ./modules/$folderName/SN.gs
                vertexShader ./modules/$folderName/SN.vs
                fragmentShader   ./modules/$folderName/SN.fs
                textureFBO stateTexture stateTexture
                texture cmap ./modules/$folderName/cmap.png
                {             
                    wrapModeS GL_CLAMP_TO_EDGE
                    wrapModeR GL_CLAMP_TO_EDGE
                    colorspace linear
                }  
                
                stateManagerVar __objectName__.SNRadMax  SNRadMax
                stateManagerVar __objectName__.SNTimeMin  SNTimeMin
                stateManagerVar __objectName__.SNTimeMax  SNTimeMax
                stateManagerVar __objectName__.SNAlpha  SNAlpha
                stateManagerVar __objectName__.SNFadeFac  SNFadeFac
                #stateManagerVar __objectName__.eventTime  eventTime
                
                glState
                {
                    UV_CULL_FACE_ENABLE false
                    UV_BLEND_ENABLE true
                    UV_DEPTH_ENABLE true
                    UV_WRITE_MASK_DEPTH false
                    UV_BLEND_FUNC GL_SRC_ALPHA GL_ONE_MINUS_SRC_ALPHA
                }
            }

        }    
    }




}"""

### SN class

In [43]:
class SN1987a():
    def __init__(self, object):
        self.object = object
        uvmod.Utility.ensurerelativepathexsists("pass0.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("state.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SN.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SN.vs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("SN.fs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("neutrino.gs",dataFolder)
        uvmod.Utility.ensurerelativepathexsists("neutrino.fs",dataFolder)
        self.cr = 1000
        self.Scale = 1

    def generatemod(self):
        self.object.setgeometry(self.object.name+"Mesh.usesconf")
        return self.object.generatemod()
    def generatefiles(self, absOutDir, relOutDir):
        fileName = self.object.name+"Mesh.usesconf"
        s = string.Template(Template)
        f = open(absOutDir+"\\"+fileName, 'w')
        if f:
            f.write(s.substitute(folderName = relOutDir,
                                 cr = self.cr,
                                 Scale = self.Scale
                                 ))
            f.close()
        uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, absOutDir)

### Object Instantiation

In [44]:
model = SN1987a(uvmod.OrbitalObject())
scene = uvmod.Scene()
parentScene = uvmod.Scene()
modinfo = uvmod.ModuleInformation()
generator = uvmod.Generator()

### Specify Settings and generate the module

In [47]:
scene.setname("SN1987a")
scene.setparent("MilkyWay")
scene.setunit(100000000000.0)
scene.setentrydist(100000.)
scene.setstaticposition(1943.83535919, 18089.69093062, -48072.2850105)

model.object.setcameraradius(0.0001)
model.object.setcoord(scene.name)
model.object.setname("SN1987a")
model.object.setguiname("/KavliLecture/Larson/SN1987a")
model.object.settargetradius(10)
model.object.showatstartup(False)
model.cr = 1000

modinfo.setname("SN1987a")
modinfo.setauthor("Aaron Geller<sup>1</sup> and Shane Larson<sup>2</sup><br />(1)Adler Planetarium,<br />(2)Northwestern University")
modinfo.cleardependencies()
modinfo.setdesc("Uniview module for SN 1987a")
#modinfo.setthumbnail("data/R0010133.JPG")
modinfo.setversion("1.0")

generator.generate("SN1987a",[scene],[model],modinfo)
uvmod.Utility.senduvcommand(model.object.name+".reload")

## Helper Functions for modifing code
*Reload Module and Shaders in Uniview*

In [168]:
uvmod.Utility.senduvcommand(model.object.name+".reload")

Unable to connect to Uniview


*Copy modified Shader files and reload*

In [24]:
from config import Settings

In [25]:
uvmod.Utility.copyfoldercontents(os.getcwd()+"\\"+dataFolder, Settings.uvcustommodulelocation+'\\'+model.object.name)
uvmod.Utility.senduvcommand(model.object.name+".reload")

Unable to connect to Uniview


### Create colormap texture

In [124]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [79]:
gradient = np.linspace(0, 1, 256)
gradient = np.vstack((gradient, gradient))

def plot_cmap(colormap, fname):
    fig=plt.imshow(gradient, aspect=1, cmap=colormap)
    plt.axis('off')
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.savefig(fname, bbox_inches='tight',pad_inches=0)

plot_cmap('plasma', 'data/cmap.png')
plot_cmap('viridis', 'data/cmapNS.png')

## SN1987a position 

*From the [wikipedia page](https://en.wikipedia.org/wiki/SN_1987A)*

In [6]:
from astropy.coordinates import SkyCoord
from astropy import units

In [8]:
RA = "05h 35m 28.03s" #right ascension
Dec= "−69d 16m 11.79s" #declination
dist = (51.4 *units.kpc).to(units.pc) #distance

coord = SkyCoord(RA, Dec, dist)

print(coord.cartesian)

(1943.83535919, 18089.69093062, -48072.2850105) pc
